# Idősoros előrejelzés gépi tanulási alapokon
## Tőzsdei előrejelzés

Az idősoros előrejelzés célja, hogy prediktáljunk jövőbeli adatokat múltbeli adatok alapján. Jelen önálló laboratórium munka során, tőzsdei előrejelzésekhez lettek készítve modellek, majd az így kapott eredmények vannak feldolgozva, minnél jobb tőzsdei technika kidolgozásához. A tőzsdei előrejelzéseket felbonthatjuk két csoportba : rövid- és hoszzú távó előrejelzések. Jelen dolgozat során rövid távú előrejelzéseket végzünk, arra vonatkozóan, hogy egy adott részvény esetén, az elkövetkezendő nap nőni vagy esni fog a részvény értéke.

A modellek elkészítéséhez és a kiértékelés megírásához python nyelvet használtam, mivel a beépített API-k nagy mértékben megkönnyítik a deep learning modellek felépítését (a Tensorflow és Keras felelős ezekért), valamint az eredmények vizuális prezentációját (ehhez például a matplotlib nyújtott sok segítséget).



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

A felhasznált adathalmaz a kaggle-n található Huge Stock Market Database. (https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs)

Ez az adathalmaz több mint hétezer részvény historikus adatait tárolja napi bontásban. Minden munkanapra megtalálható a részvény nyitó, záró, legalacsonyabb és legmagasabb ára. 

In [ ]:
df = pd.read_csv(os.path.join('../input/price-volume-data-for-all-us-stocks-etfs/Stocks','cl.us.txt'),delimiter=',',usecols=['Date','Open','High','Low','Close'])
print('Loaded data from the Kaggle repository')

df.head()

Mivel a teljes adathalmazon nagyon sok idő lenne a tanitást elvégezni, és túlságosan régi adatok nagyon kevésbé befolyásolnák a modell végső eredményét a felépítése miatt, ezért csak 5 évnyi adat lesz feldolgozva.

In [ ]:
df = df[df['Date'].str.contains("2012") | df['Date'].str.contains("2013") | df['Date'].str.contains("2014") | df['Date'].str.contains("2015") | df['Date'].str.contains("2016") ]
df.head()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),(df['Low']+df['High'])/2.0)
plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.show()

In [ ]:
TEST_INTERVAL = 200
TIMESTAMP = 50
EPOCH = 200
BATCH_SIZE = 32
NUM_OF_RUNS = 10

Tanítás és tesztelés során, egy részvény napi ára alatt a napi mediánt fogjuk érteni, azaz a napi legalacsonyabb és napi legmagasabb érték átlagát. 

Az adathalmazt 2 részre bontjuk: tanító és tesztelő adathalmaz. Az utolsó 200 napi adatot teszteljük le a modell hatékonyságát, minden előtte lévő adat a tanításhoz lesz használva.

In [ ]:
high_prices = df.loc[:,'High'].to_numpy()
low_prices = df.loc[:,'Low'].to_numpy()
mid_prices = (high_prices+low_prices)/2.0

print(mid_prices)

data_num = len(mid_prices)
data_num_train = data_num - TEST_INTERVAL
train_data = mid_prices[:data_num_train]
test_data = mid_prices[data_num_train-TIMESTAMP:]

Standardizáljuk a tesztelő és tanító adathalmaz elemeit. Ennek célja, hogy az adatok elosztása a normál elosztást kövesse, amely könnyeben feldolgozható a neurális hálóm számára.

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
train_data = train_data.reshape(-1,1)
train_data_scaled = sc.fit_transform(train_data)


In [ ]:
print(train_data_scaled)

In [ ]:
X_train = []
y_train = []
for i in range(TIMESTAMP, data_num_train):
    X_train.append(train_data_scaled[i-TIMESTAMP:i, 0])
    y_train.append(train_data_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
real_stock_price = test_data[TIMESTAMP:]
predicted_stock_prices = np.zeros((NUM_OF_RUNS,TEST_INTERVAL))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import SimpleRNN
from keras.layers import GRU

<img src='https://www.researchgate.net/profile/Xiaofeng-Yuan-4/publication/331421650/figure/fig2/AS:771405641695233@1560928845927/The-structure-of-the-LSTM-unit.png'/>


Az LSTM (Long Short-Term Memory) egy rekurrens neurális háló, tipikusan mély tanulás során használatos. A fenti ábrán látható az LSTM egy egysége, amelynek három bemenete van: x(t) adott időpillanatbeli bemenet, h(t-1) az adott időpillanatig kiszámolt eredmény (hidden statenak is nevezik), a c(t-1) pedig az előző időpillantig kiszámolt cell state, amely memóriaként szolgál, ehhez adódnak vagy vonódnak ki adatok. A cell state adatait a különböző kapuk befolyásolják, amelyek a következők:


*   forget gate: eldönti, hogy a hidden state és az adott időpillantban kapott adatokból mennyit érdemes megtartani
*   input gate: a cell state updateolására szolgál. Eldönti, hogy mely adatokat érdemes frissíteni
*  output gate: a következő h(t) kiszámolására szolgál



In [ ]:
def create_lstm_model():
    model = Sequential()

    model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))

    model.add(Dense(units = 1))

    model.summary()
    
    return model

In [ ]:
'''
def create_rnn_model():
    model = Sequential()
    model.add(SimpleRNN(32, return_sequences=True))
    model.add(SimpleRNN(32, return_sequences=True))
    model.add(SimpleRNN(32, return_sequences=True))
    model.add(SimpleRNN(32))
    model.add(Dense(1))
    
    return model
  '''

<img src='https://paperswithcode.com/media/methods/780px-Gated_Recurrent_Unit_type_1.svg.png' />

A GRU, egy az LSTM-hez hasonlító neurális háló. A különbség az LSTM és a GRU között, hogy a GRU-ban nincs cell state, ez a hidden state része a továbbiakban. A másik különbség az LSTM-hez képest, hogy 3 kapu helyett csak 2 kapuja van:


*   update gate (z[t]): az LSTM-beli forget és update gate megfelelője. Eldönti, hogy mely adatok megtartása releváns 
*   reset gate (r[t]): eldönti, hogy a múltbeli adatokból mennyit felejtsen el a háló

Mivel az LSTM és GRU modellt is kipróbálva, nagyobb százalékban hozott a GRU jobb eredményeket, így a továbbiakban a lent látható kódban létrehozott GRU modellt fogjuk használni. Felépítése


*   Öt GRU réteg
*   ELső négy réteg 50 egységnyi GRU-t tartalmaz
*   Utolsó réteg, mely a kimeneti réteg, 1 egységnyi GRU-t tartalmaz
*   Túltanulás elkerülése érdekében 0.2-es dropout használunk az első négy rétegen





In [ ]:
def create_GRU_model():
    regressorGRU = Sequential()
    # First GRU layer with Dropout regularisation
    regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Second GRU layer
    regressorGRU.add(GRU(units=50, return_sequences=True, activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    
    # Third GRU layer
    regressorGRU.add(GRU(units=50, return_sequences=True, activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Fourth GRU layer
    regressorGRU.add(GRU(units=50, activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # The output layer
    regressorGRU.add(Dense(units=1))
    return regressorGRU

Mivel a GRU modell lefutása nem determinisztikus, ezért ahhoz, hogy valós statisztikát tudjunk készíteni az eredményből, minden egyes részvényre többször is lefuttatjuk a modellt. Ennek az értéke a NUM_OF_RUNS konstansban található, melynek 10-es értéket adtunk meg. Loss függvénynek MSE-e használnuk, azaz a valóst értéktől való különbségek négyzetének átlagát.

A predicted_stock_price változóban eltároljuk a futás eredményét (azaz a predikciót), ahhoz a következőkben ezt fel tudjuk dolgozni.

In [ ]:
for i in range(NUM_OF_RUNS):
    print(i, ". RUN")
    
    model = create_GRU_model()

    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    model.fit(X_train, y_train, epochs = EPOCH, batch_size = BATCH_SIZE)
    
    data_num_test = len(test_data)
    test_data = test_data.reshape(-1,1)
    test_data_scaled = sc.fit_transform(test_data)
    X_test = []
    for k in range(TIMESTAMP, data_num_test):
        X_test.append(test_data_scaled[k-TIMESTAMP:k, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    
    print(len(predicted_stock_price))
    
    for j in range(TEST_INTERVAL):
        predicted_stock_prices[i][j] = predicted_stock_price[j]

In [ ]:
#data_num_test = len(test_data)
#test_data = test_data.reshape(-1,1)

In [ ]:
#real_stock_price = test_data[TIMESTAMP:]

In [ ]:
#test_data_scaled = sc.fit_transform(test_data)

In [ ]:
'''
X_test = []
for i in range(TIMESTAMP, data_num_test):
    X_test.append(test_data_scaled[i-TIMESTAMP:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
'''

In [ ]:
'''
plt.plot(real_stock_price, color = 'black', label = 'Stock Price')
plt.plot(predicted_stock_price, color = 'green', label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
'''

In [ ]:
'''correct_growth_per_real = 0
correct_growth_per_predicted = 0
for i in range(1, TEST_INTERVAL):
    if(predicted_stock_price[i]>real_stock_price[i-1] and real_stock_price[i]>real_stock_price[i-1]):
        correct_growth_per_real += 1
    elif(predicted_stock_price[i]<real_stock_price[i-1] and real_stock_price[i]<real_stock_price[i-1]):
        correct_growth_per_real += 1
    if(predicted_stock_price[i]>predicted_stock_price[i-1] and real_stock_price[i]>real_stock_price[i-1]):
        correct_growth_per_predicted +=1
    elif(predicted_stock_price[i]<predicted_stock_price[i-1] and real_stock_price[i]<real_stock_price[i-1]):
        correct_growth_per_predicted +=1
        '''

In [ ]:
#print("Correct grow per real: ", correct_growth_per_real/TEST_INTERVAL)
#print("Correct grow per predicted: ", correct_growth_per_predicted/TEST_INTERVAL)

In [ ]:
real_stock_price_profit_per_day = []
for i in range(1, TEST_INTERVAL):
    real_stock_price_profit_per_day.append((real_stock_price[i] - real_stock_price[0])/real_stock_price[0] * 100)
    
predicted_stock_price_profits_per_day = [[0 for j in range(TEST_INTERVAL)] for i in range(NUM_OF_RUNS)]


In [ ]:
avg_profit = 0

A modell kimeneteit lementjük, ahhoz, hogy elemezzük, hogy segítségükkel milyen porfitot tudunk realizálni. A következő notebook (evaluation.ipynb) az eredmények kiértékeléséről fog szólni


In [ ]:
np.savetxt("cl_pred.csv", predicted_stock_prices, delimiter=",")

In [ ]:
'''
def calculateProfit(diffStock):
    all_profit = 0
    min_profit = 10000000
    max_profit = -10000000
    for j in range(NUM_OF_RUNS):
        predicted_stock_price = predicted_stock_prices[j]
        original_stock_price = real_stock_price[0]

        bought_stock_price = real_stock_price[0]
        stock_price_at_selling = 0
        stock_buys_total = real_stock_price[0]
        stock_sells_total = 0
        profit = 0

        for i in range(1, TEST_INTERVAL):
            if(predicted_stock_price[i]>diffStock[i-1]):
                if(bought_stock_price==0):
                  print("Buying at ", i, " day:", real_stock_price[i])
                  bought_stock_price = real_stock_price[i]
                  stock_buys_total = stock_buys_total + real_stock_price[i]
            else:
                if(bought_stock_price!=0):
                  print("Selling at ", i, " day:", real_stock_price[i], "     We made:", real_stock_price[i]-bought_stock_price)
                  bought_stock_price = 0
                  stock_price_at_selling = real_stock_price[i]
                  stock_sells_total = stock_sells_total + real_stock_price[i]
            profit = stock_sells_total - stock_buys_total
            if(bought_stock_price==0):
                predicted_stock_price_profits_per_day[j][i] = (profit)/original_stock_price * 100
            else:
                predicted_stock_price_profits_per_day[j][i] = (profit+real_stock_price[i])/original_stock_price * 100

        print("Money spent on buying:", stock_buys_total)    
        print("Money received on selling:", stock_sells_total) 
        print("cash profit:", profit)
        print("actual stock price:", real_stock_price[TEST_INTERVAL-1]) 

        if(bought_stock_price==0):
            print("We made ", profit, " from ", original_stock_price)
            act_profit = (profit)/original_stock_price * 100
            print("Profit:", act_profit, "%")
            all_profit = all_profit + act_profit
            if(act_profit>max_profit):
                max_profit = act_profit
            if(act_profit<min_profit):
                min_profit = act_profit
            print("Stock growth:", real_stock_price[TEST_INTERVAL-1], " ", (real_stock_price[TEST_INTERVAL-1] - original_stock_price)/original_stock_price * 100, "%")
        else:
            print("We made ", profit+real_stock_price[TEST_INTERVAL-1], " from ", original_stock_price)
            act_profit = (profit+real_stock_price[TEST_INTERVAL-1])/original_stock_price * 100
            print("Profit:", (profit+real_stock_price[TEST_INTERVAL-1])/original_stock_price * 100, "%")
            all_profit = all_profit + act_profit
            if(act_profit>max_profit):
                max_profit = act_profit
            if(act_profit<min_profit):
                min_profit = act_profit
            print("Stock growth:", real_stock_price[TEST_INTERVAL-1], " ", (real_stock_price[TEST_INTERVAL-1] - original_stock_price)/original_stock_price * 100, "%")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
    avg_profit = all_profit / NUM_OF_RUNS 
    
    print("AVG profit:", avg_profit)
    print("MAX profit:", max_profit)
    print("MIN profit:", min_profit)
'''

In [ ]:
'''
def calculateProfitUsingAll(diffStock):
    STOCK_CAP = 10
    STRONG_SELL_MIN = 0
    SELL_MIN = 2
    STALL_MIN = 4
    BUY_MIN = 6
    STRONG_BUY_MIN = 8
    NORMAL_BUY_SELL = 1
    STRONG_BUY_SELL = 3
    
    bought_stocks = 1
    money_spent = real_stock_price[0]
    money_received = 0
    for i in range(1, TEST_INTERVAL):
        BUY_PRED_NUM = 0
        for j in range(NUM_OF_RUNS):
            predicted_stock_price = predicted_stock_prices[j][i]
            if(predicted_stock_price>diffStock[i-1]):
                BUY_PRED_NUM = BUY_PRED_NUM + 1
        if(BUY_PRED_NUM<SELL_MIN):
            if(bought_stocks > 0):
                if(bought_stocks > STRONG_BUY_SELL):
                    bought_stocks = bought_stocks - 3
                    money_received = 3 * real_stock_price[i]
                    print(i, ". day: STRONG SELL. We sold 3 stocks. Current stock number:", bought_stocks)
                else:
                    bs_temp = bought_stocks
                    bought_stocks = bought_stocks - bs_temp
                    money_received = bs_temp * real_stock_price[i]
                    print(i, ". day: STRONG SELL. We sold ", bs_temp, " stocks. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<STALL_MIN):
            if(bought_stocks > 0):
                bought_stocks = bought_stocks - 1
                money_received = 1 * real_stock_price[i]
                print(i, ". day: SELL. We sold 1 stock. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<BUY_MIN):
            print(i, ". day: STALL. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<STRONG_BUY_MIN):
            if(bought_stocks < STOCK_CAP):
                bought_stocks = bought_stocks + 1
                money_spent = 1 * real_stock_price[i]
                print(i, ". day: BUY. We bought 1 stock. Current stock number:", bought_stocks)
        else:
            if(bought_stocks < STOCK_CAP):
                stocks_to_cap = STOCK_CAP - bought_stocks
                if(stocks_to_cap < STRONG_BUY_SELL):
                    bought_stocks = bought_stocks + stocks_to_cap
                    money_spent = stocks_to_cap * real_stock_price[i]
                    print(i, ". day: BUY. We bought ", stocks_to_cap, " stock. Current stock number:", bought_stocks)
                else:
                    bought_stocks = bought_stocks + 3
                    money_spent = 3 * real_stock_price[i]
                    print(i, ". day: STRONG BUY. We bought 3 stock. Current stock number:", bought_stocks)
''' 

In [ ]:
#print("Comparing to predicted stock price:")
#calculateProfit(predicted_stock_price)

In [ ]:
'''
print("Comparing to real stock price:")
calculateProfit(real_stock_price)
'''

In [ ]:
'''
plt.figure(figsize = (18,9))
for i in range(NUM_OF_RUNS):
    plt.plot(predicted_stock_price_profits_per_day[i], color = np.random.rand(3,)
             , label = 'Stock Price Profit Prediction ' + str(i))
plt.plot(real_stock_price_profit_per_day, color = 'black', label = 'Stock Price Profit')
plt.title('Stock Price Profit Per Day')
plt.xlabel('Time')
plt.ylabel('Profit')
plt.legend()
plt.show()
'''

In [ ]:
#calculateProfitUsingAll(real_stock_price)